In [57]:
from transformers import AutoTokenizer
from scipy.special import softmax
import os
import tweepy 
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import warnings
import time 
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()


In [58]:
import os
import tweepy
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from dotenv import load_dotenv
import json


class api():

    def __init__(self, bearer) -> None:
        self.client = tweepy.Client(bearer_token=bearer,wait_on_rate_limit=True)

    def scrape(self, query, num_tweets,runs,start_date):
        start_date = datetime.strptime(start_date,'%Y-%m-%d')
        # Creating DataFrame using pandas



        # tweets  = self.client.search_all_tweets(query=query,
        #                                         tweet_fields = ['author_id','created_at','text','source','lang','geo'],
        #                                         user_fields = ['name','username','location','verified','created_at','description','public_metrics'],
        #                                         # expansions = ['geo.place_id', 'author_id','in_reply_to_user_id'],
        #                                         expansions = [ 'author_id'],
        #                                         place_fields = ['country','country_code'] ,
        #                                         max_results=num_tweets)
        pag = tweepy.Paginator(
            self.client.search_all_tweets,
            query=query,
            start_time = start_date,
            tweet_fields=[
                'author_id', 'created_at', 'text', 'source', 'lang', 'geo',
                'public_metrics', 'possibly_sensitive'
            ],
            user_fields=[
                'name', 'username', 'profile_image_url', 'location',
                'verified', 'created_at', 'description', 'public_metrics'
            ],
            expansions=['geo.place_id', 'author_id', 'in_reply_to_user_id'],
            place_fields=['country', 'country_code'],
            max_results=num_tweets)

        # print({u['id']:u.created_at for u in pag.data.includes['users']})
        # list_tweets = [tweet for tweet in tweets[0]]
        i = 1
        # places = {p["id"]: p for p in pag.includes['places']}

        for k,page in enumerate(pag):
                db = pd.DataFrame(columns=[
                                'tweet_text', 'hashtags', 'sensitivity', 'tweet_likes',
                                'num_retweets', 'reply_count', 'quote_count', 'lang',
                                'replied_user_id' , 'retweeters',
                                'mentions', 'location', 'user_created_at', 'user_id', 'username',
                                'desc', 'author_followers_count', 'author_following_count',
                                'verified', 'profile_pic_url', 'user_location'
                                ])
                # print(page.includes.items())
                user_fields = {u['id']: u for u in page.includes['users']}
                places = {p["id"]: p for p in page.includes['places']}
                list_tweets = [tweet for tweet in page[0]]
                print(len(list_tweets))
                for tweet in list_tweets:

                        #tweet attributes
                        sensitivity = tweet.possibly_sensitive

                        tweet_likes = tweet.public_metrics['like_count']
                        num_retweets = tweet.public_metrics['retweet_count']
                        reply_count = tweet.public_metrics['reply_count']
                        quote_count = tweet.public_metrics['quote_count']
                        lang = tweet.lang
                        replied_user_id = tweet.in_reply_to_user_id
                        id = tweet.author_id

                        retweeters = self.client.get_retweeters(tweet.id) if reply_count > 0 else 0 

                        raw_names = re.findall(
                        r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)",
                        tweet.text)
                        raw_names = [x.lower() for x in raw_names]
                        if raw_names:
                                mentions = list(dict.fromkeys(raw_names))
                                # raw_names = list(dict.fromkeys(raw_names))
                                # mention_data = self.client.get_users(
                                        # usernames=raw_names).data
                                # try :   
                                #                 mentions = [
                                #                 x.id for x in mention_data if mention_data is not None
                                #         ]
                                # except Exception as e :
                                #         print( e)
                                #         print(mention_data)
                        
                        else:
                                mentions = []
                        if places[tweet.geo['place_id']]:
                                location = places[tweet.geo['place_id']].full_name

                        #user attributes
                        user_created_at = user_fields[id].created_at
                        user_id = tweet.author_id
                        user_data = user_fields[user_id]
                        username = user_data.username
                        desc = user_data.description
                        author_followers_count = user_data.public_metrics['followers_count']
                        author_following_count = user_data.public_metrics['following_count']
                        verified = user_data.verified
                        profile_pic_url = user_data.profile_image_url
                        user_location = user_data.location
                        date = tweet.created_at
                        hashtags = re.findall(r"#(\w+)", tweet.text)
                        ith_tweet = [
                        tweet.text, hashtags, sensitivity, tweet_likes,
                        num_retweets, reply_count, quote_count, lang,
                        replied_user_id, retweeters,
                        mentions, location, user_created_at, user_id, username,
                        desc, author_followers_count, author_following_count,
                        verified, profile_pic_url, user_location
                        ]

                        db.loc[len(db)] = ith_tweet
                        i = i + 1
                print(k,'k')
                db.to_csv('scraped_tweets.csv', mode='a', header=False) if k == 0 else db.to_csv('scraped_tweets.csv')
                if k == runs:
                        break 
                time.sleep(1)


load_dotenv()
from datetime import datetime

use = api(os.getenv('bearer'))
use.scrape('#Iran place_country:IR', 500,3,'2022-06-01')


494


Rate limit exceeded. Sleeping for 890 seconds.


0 k
470


Rate limit exceeded. Sleeping for 888 seconds.


1 k
470


Rate limit exceeded. Sleeping for 888 seconds.


2 k
456


Rate limit exceeded. Sleeping for 888 seconds.


3 k


,tweet_text,hashtags,sensitivity,tweet_likes,num_retweets,reply_count,quote_count,lang,replied_user_id,retweeters,...,location,user_created_at,user_id,username,desc,author_followers_count,author_following_count,verified,profile_pic_url,user_location
0,"Dec 7—#Tehran, #Iran \nFour snipers were seen ...","[Tehran, Iran, IranRevolution]",False,23,18,1,0,en,None,"([(id, name, username), (id, name, username), ...",...,برج باغ ترانه/Taraneh residence,2013-04-27 19:08:19+00:00,1385266495,journalistsay,I believe in Newton's 3rd law. I fight against...,2654,5004,False,https://pbs.twimg.com/profile_images/157134836...,192.168.1.1
1,"Dec 7—#Tehran, #Iran \nLocals near Azadi Squar...","[Tehran, Iran, IranRevolution]",False,28,14,2,0,en,None,"([(id, name, username), (id, name, username), ...",...,République islamique d'Iran,2013-04-27 19:08:19+00:00,1385266495,journalistsay,I believe in Newton's 3rd law. I fight against...,2654,5004,False,https://pbs.twimg.com/profile_images/157134836...,192.168.1.1
2,#Iran to host Junior World Kabaddi Championshi...,[Iran],False,4,0,0,0,en,None,0,...,République islamique d'Iran,2016-12-26 07:03:39+00:00,813279088865644544,IrnaEnglish,Islamic Republic News Agency (IRNA)\n\nOfficia...,26832,38,True,https://pbs.twimg.com/profile_images/158751238...,Around the Globe
3,Chair of the Senate Foreign Relations Committe...,"[Iran, Iranian, IranRevoIution]",False,5,3,0,0,en,None,0,...,برج باغ ترانه/Taraneh residence,2013-04-27 19:08:19+00:00,1385266495,journalistsay,I believe in Newton's 3rd law. I fight against...,2654,5004,False,https://pbs.twimg.com/profile_images/157134836...,192.168.1.1
4,A student's response to regime President Ebrah...,"[Tehran, IranRevolution, Iran]",False,12,7,0,2,en,1385266495,0,...,برج باغ ترانه/Taraneh residence,2013-04-27 19:08:19+00:00,1385266495,journalistsay,I believe in Newton's 3rd law. I fight against...,2654,5004,False,https://pbs.twimg.com/profile_images/157134836...,192.168.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,@elonmusk Please please be our voice....\nInte...,"[Iran, TORTURE, ISLAMIC_GILEAD, GILEAD, MahsaA...",False,0,0,0,0,en,44196397,0,...,République islamique d'Iran,2011-11-28 19:09:59+00:00,423636948,hooman7272,(INTJ_A) 🙂 Music Meaning With #ShahinNajafi 😈 ...,1292,629,False,https://pbs.twimg.com/profile_images/157122770...,فاک آباد
1886,@elonmusk Please please be our voice....\nInte...,"[Iran, TORTURE, ISLAMIC_GILEAD, GILEAD, MahsaA...",False,0,0,0,0,en,44196397,0,...,République islamique d'Iran,2011-11-28 19:09:59+00:00,423636948,hooman7272,(INTJ_A) 🙂 Music Meaning With #ShahinNajafi 😈 ...,1292,629,False,https://pbs.twimg.com/profile_images/157122770...,فاک آباد
1887,@POTUS Please please be our voice....\nInterne...,"[Iran, TORTURE, ISLAMIC_GILEAD, GILEAD, MahsaA...",False,0,0,0,0,en,1349149096909668363,0,...,République islamique d'Iran,2011-11-28 19:09:59+00:00,423636948,hooman7272,(INTJ_A) 🙂 Music Meaning With #ShahinNajafi 😈 ...,1292,629,False,https://pbs.twimg.com/profile_images/157122770...,فاک آباد
1888,@POTUS Please please be our voice....\nInterne...,"[Iran, TORTURE, ISLAMIC_GILEAD, GILEAD]",False,0,0,0,0,en,1349149096909668363,0,...,République islamique d'Iran,2011-11-28 19:09:59+00:00,423636948,hooman7272,(INTJ_A) 🙂 Music Meaning With #ShahinNajafi 😈 ...,1292,629,False,https://pbs.twimg.com/profile_images/157122770...,فاک آباد


In [ ]:
class Sentiment():
    def __init__(self,english_model,english_tokenizer,farsi_model,farsi_tokenizer) -> None:
        self.english_model = english_model
        self.english_tokenizer = english_tokenizer
        self.farsi_model = farsi_model
        self.farsi_tokenizer = farsi_tokenizer
    def check_language(self,s): 
        s2 = re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', ' ',s).replace('\n',' ')
        s2 = re.sub(' +', ' ', s2)
        prop = len(s2)/len(s)
        if prop > .1: 
            input_ids = self.farsi_tokenizer.encode(s2, return_tensors="pt")
            res = self.farsi_model.generate(input_ids,max_length=300)
            output = self.farsi_tokenizer.batch_decode(res, skip_special_tokens=True)
            print(res)
            return output[0]
        else: 
            text = self.preprocess_english(s)
            encoded_input = self.english_tokenizer(text, return_tensors='pt')
            output = self.english_model(**encoded_input)
            scores = output[0][0].detach().numpy()
            scores = softmax(scores)
            ranking = np.argsort(scores)
            ranking = ranking[::-1]
            out = labels[np.argmax(ranking)]
            return out

    def analyze(self,df:pd.DataFrame,target_col='text',output_col = 'sentiment'):
        df[output_col] = df.text.apply(lambda x: self.check_language(x))
        return df 
    def preprocess_english(self,text):
        new_text = []
    
    
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)           
            

In [ ]:

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer_english  = AutoTokenizer.from_pretrained(MODEL)
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
model_english = AutoModelForSequenceClassification.from_pretrained(MODEL)


tokenizer_farsi = AutoTokenizer.from_pretrained("persiannlp/mt5-base-parsinlu-sentiment-analysis")

model_farsi = AutoModelForSeq2SeqLM.from_pretrained("persiannlp/mt5-base-parsinlu-sentiment-analysis")

sent = Sentiment(model_english,tokenizer_english,model_farsi,tokenizer_farsi)
sent.analyze(pd.read_csv('scraped_tweets.csv'))

/Users/emilianopenaloza/mambaforge/envs/weldon/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


tensor([[    0,   259, 32588,     1]])
tensor([[    0,   259, 32588,     1]])
tensor([[    0,   375, 34225, 17385,   345,     1]])
tensor([[    0,   375, 34225, 17385,   345,     1]])
tensor([[    0,   375, 34225, 17385,   345,     1]])
tensor([[    0,   259, 32588,     1]])


,date,username,author_id,mentions,hashtags,location,text,sentiment
0,2022-12-06 15:32:32+00:00,amir_hosein78gh,1379494022763278337,[],"['حمایت_انقلابی', 'Iran', 'IranRevolution', 'م...",République islamique d'Iran,فقط شاخ شدن بالا کجا و عکس پایین کجا😂😂😂😂\n#حما...,negative
1,2022-12-06 14:43:11+00:00,yaserimanii,1260455064352821248,[],"['زاکانی', 'دانشجو', 'انقلاب', 'براي_ایران', '...",République islamique d'Iran,پاسخ جالب #زاکانی با برخی دانشجویان معترض\n #د...,negative
2,2022-12-06 06:47:42+00:00,godofwaaarr,1579598708563496973,[],"['كريستيانو_رونالدو', 'Iran', 'مهسا_ایمنی', 'ک...",République islamique d'Iran,اینها چرا اینقدر باورها و قیافه هاشون کیریه\n\...,no sentiment expressed
3,2022-12-05 21:01:25+00:00,journalistsay,1385266495,[],"['Rasht', 'Iran', 'IranRevolution']",République islamique d'Iran,"Dec 5—#Rasht, N #Iran \nSecurity forces opened...",positive
4,2022-12-05 20:41:06+00:00,journalistsay,1385266495,[],"['Tehran', 'Iran', 'IranRevolution']",Balaye Ekbatan,"Dec 5—#Tehran, central #Iran \nMany locals are...",positive
5,2022-12-05 18:03:58+00:00,RezayatHadis,1551256587129573378,[],"['Iran', 'pianist', 'مهسا_امینی']",République islamique d'Iran,#Iran\n#pianist \n#مهسا_امینی https://t.co/rc2...,no sentiment expressed
6,2022-12-05 17:43:04+00:00,journalistsay,1385266495,[],"['Tehran', 'Iran', 'IranRevolution']",Sadeghiyeh Metro Station | ایستگاه مترو صادقیه,"Dec 5—#Tehran, #Iran \nThe protesters in the c...",positive
7,2022-12-05 11:27:49+00:00,salar__bagheri,1565399154121129986,[],"['اعتصابات_سراسری', 'ایران', 'Iran']",République islamique d'Iran,رییس محترم قوه قضاییه توصیه ای رو به والدین کر...,no sentiment expressed
8,2022-12-04 23:25:24+00:00,alishahbaziart,1236046969140850688,[],"['مهسا_امینی', 'زن_زندگی_آزادی', 'ايران_آزاد',...",République islamique d'Iran,ظلم نمى ماند . . .\n.\n#مهسا_امینی #زن_زندگی_...,negative
9,2022-12-04 21:24:48+00:00,marzprad,1580317621412511744,[],['Iran'],République islamique d'Iran,Wanna know one of many truths related to the I...,positive


Ideally we have the following graphs: 

Number of tweets per hashtag per day 
line chart 
